In [1]:
import sys
sys.path.append("../../")

In [2]:
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.script import *
import fastai.vision.learner as fvl
from fastai.vision.learner import cnn_config
from fastai.vision.models.unet import DynamicUnet

from skimage import filters
from skimage.util import random_noise
from functools import partial
from model.metrics import *
from data.utils import load_img
import libtiff
from skimage.measure import compare_ssim, compare_psnr
from csbdeep.utils import normalize


In [3]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/')

In [4]:
img_files = sorted(list(data_pth.glob('*.tif')))
img_files

[PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/05_hr.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/05_lr_acquired.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/05_pred_emrealworld_em_AG_D_sameas_preprint3.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/42_hr.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/42_lr_acquired.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/42_pred_emrealworld_em_AG_D_sameas_preprint3.tif')]

In [5]:
preds = sorted([img_files[2], img_files[5]])
targets = sorted([img_files[0], img_files[3]])
inputs = sorted([img_files[1], img_files[4]])
preds

[PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/05_pred_emrealworld_em_AG_D_sameas_preprint3.tif'),
 PosixPath('/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/emrealworld_em_AG_D_sameas_preprint/42_pred_emrealworld_em_AG_D_sameas_preprint3.tif')]

In [6]:
pred1 = PIL.Image.open(preds[0])
targ1 = PIL.Image.open(targets[0])
inp1 = PIL.Image.open(inputs[0])
pred2 = PIL.Image.open(preds[1])
targ2 = PIL.Image.open(targets[1])
inp2 = PIL.Image.open(inputs[1])

In [9]:
pred1_arr = np.array(pred1)
targ1_arr = np.array(targ1)
inp1_arr = np.array(inp1)
pred2_arr = np.array(pred2)
targ2_arr = np.array(targ2)
inp2_arr = np.array(inp2)
targ2_arr.shape

(500, 500)

In [32]:
compare_psnr(targ1_arr, pred1_arr, data_range=255.)

27.178023273248346

In [33]:
compare_ssim(targ1_arr, pred1_arr, data_range=255.)

0.5799516770354562

In [34]:
compare_psnr(targ2_arr, pred2_arr, data_range=255.)

27.384507801570003

In [35]:
compare_ssim(targ2_arr, pred2_arr, data_range=255.)

0.6107373728964813

In [7]:
def gaussian(window_size, sigma):
    gauss = torch.Tensor([exp(-(x - window_size//2)**2/float(2*sigma**2)) for x in range(window_size)])
    return gauss/gauss.sum()
def create_window(window_size, channel):
    _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).float().unsqueeze(0).unsqueeze(0)
    window = Variable(_2D_window.expand(channel, 1, window_size, window_size).contiguous())
    return window
def _ssim(img1, img2, window, window_size, channel, data_range=255., padding=False, size_average = True):
    if padding:
        padding = window_size // 2
    mu1 = F.conv2d(img1, window, padding = int(padding), groups = channel)
    mu2 = F.conv2d(img2, window, padding = int(padding), groups = channel)
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1*mu2
    sigma1_sq = F.conv2d(img1*img1, window, padding = int(padding), groups = channel) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding = int(padding), groups = channel) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding = int(padding), groups = channel) - mu1_mu2
    C1 = (0.01*data_range)**2
    C2 = (0.03*data_range)**2
    ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2))/((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)
def ssim(img1, img2, window_size = 11, data_range=255, padding = False, size_average = True):
    (_, channel, _, _) = img1.size()
    window = create_window(window_size, channel)
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.type_as(img1)
    return _ssim(img1, img2, window, window_size, channel, data_range, padding, size_average)
def psnr(pred, targs, data_range=255):
    mse = F.mse_loss(pred, targs)
    return 20 * torch.log10(data_range / torch.sqrt(mse))

In [25]:
psnr(torch.FloatTensor(pred1_arr).cuda(), torch.FloatTensor(targ1_arr).cuda(), data_range=255.)

tensor(27.1780, device='cuda:1')

In [10]:
ssim(torch.cuda.IntTensor(pred1_arr).reshape((1,1,500,500)), 
     torch.cuda.IntTensor(targ1_arr).reshape((1,1,500,500)), 
     data_range=255.)

RuntimeError: getCudnnDataType() not supported for Int

In [27]:
psnr(torch.FloatTensor(pred2_arr).cuda(), torch.FloatTensor(targ2_arr).cuda(), data_range=255.)

tensor(27.3845, device='cuda:1')

In [32]:
ssim(torch.FloatTensor(pred2_arr).reshape((1,1,500,500)).cuda(), 
     torch.FloatTensor(targ2_arr).reshape((1,1,500,500)).cuda(), 
     data_range=255.)

tensor(0.6090, device='cuda:1')